## Imports

#### Run the following cells before navigating through the notebook

In [ ]:
%env SM_FRAMEWORK=tf.keras

In [ ]:
#Optional Code to reset GPU
from numba import cuda 
device = cuda.get_current_device()
print(device)
device.reset()

In [ ]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
import warnings

def function_that_warns():
    warnings.warn("deprecated", DeprecationWarning)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    function_that_warns()  # this will not show a warning

In [ ]:
from patchify import patchify
import os
import numpy as np
import random
import sys
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import PIL
from PIL import Image
import glob
import cv2
from patchify import patchify, unpatchify
import seaborn as sns

np.set_printoptions(threshold=sys.maxsize)

## Training Phase # 01

In [ ]:
# Inititate Unet using Segmentation Library
import segmentation_models as sm
from segmentation_models import Unet
BACKBONE = 'vgg16'
model = sm.Unet(BACKBONE, encoder_weights='imagenet')
#model.summary()

In [ ]:
# Creating Images and masks path and deleting an images whose masks didnt exist
count = 0
train_ids_paths=[]
train_masks_path=[]

# Change the path to the image database !!
base_path = ['mcoustat/TRAIN_1','mcoustat/TRAIN_2','mcoustat/TRAIN_3']

for base in base_path:
    total_maps= os.listdir(base)
    with tf.device('/gpu:0'):
      for i in total_maps:
        t = sorted(glob.glob(os.path.join(base, i, 'images', '*.png')))
        m = sorted(glob.glob(os.path.join(base, i, 'labels','Undefined', '*.png')))
        total_imgs=[]
        total_masks=[]

        for item in t:
            t1=item.split('/')
            total_imgs.append(t1[-1])

        for item in m:
            m1=item.split('/')
            total_masks.append(m1[-1]) 

        list_difference = []
        for item in total_imgs:
            if item not in total_masks:
                list_difference.append(item)

        for item in list_difference:
            toRemove= os.path.join(base, i, 'images', item)
            t.remove(toRemove)


        if len(t) == len(m):
            train_ids_paths +=t
            train_masks_path += m
        else:
            print( 'number of images',len(t), i) 
            print( 'number of masks',len(m), i)
    print(len(train_ids_paths), len(train_masks_path))


In [ ]:
# Print Total Images and masks
print(len(train_ids_paths), len(train_masks_path))

In [ ]:
#Reading Images and masks and creating tensors respectively

from tqdm import tqdm
Masks=[]
Images=[]  
count = 0 
with tf.device('/gpu:0'):     
  for img_path,mask_path in tqdm(zip(train_ids_paths, train_masks_path),total=len(train_ids_paths),position=0, leave=False):
    img = Image.open(img_path)
    Images.append(np.array(img))
    mask= cv2.imread(mask_path,2)
    mask= np.array(mask)
    mask = np.expand_dims(mask,-1)
    mask[mask>0]=1
    Masks.append(mask)
    count = count +1
print(len(Images), len(Masks))

In [ ]:
#Converting Images and masks into numpy array
Images=np.array(Images)
Masks=np.array(Masks)
Images.shape, Masks.shape

In [ ]:
#Shuffling images and masks to get rid of overfitting
from sklearn.utils import shuffle
Images, Masks=shuffle(Images, Masks, random_state=42)

In [ ]:
#Display 10 images and respective maps
count = 0
for i in range(len(Images)):
  if count < 5:
      fig = plt.figure(figsize=(10, 7))
      rows,columns=1,2
      fig.add_subplot(rows, columns, 1)
      toShow=(1)*4  
      # showing image
      plt.imshow(Images[i])
      plt.axis('off')
      plt.title("Image")

      # Adds a subplot at the 2nd position
      fig.add_subplot(rows, columns, 2)
      plt.imshow(Masks[i][:,:,0], cmap='gray')
      plt.axis('off')
      plt.title("Mask")
  count +=1

print (count)

#### Selecting Data for Train and Testing

In [ ]:
#Creating copy for images and masks
Images_1 = Images.copy()
Masks_1 = Masks.copy()

In [ ]:
# Out of 34000 images , select 14000 images for testing
test_Images=Images_1[20000:30000, :,:,:]
test_Masks= Masks_1[20000:30000, :,:,:]

# To check result the number of selected images should be thousand, because sometimes GPU error comes when using 14000 images.
# If GPU error comes in line 25 use following lines

# test_Images=Images_1[20000:21000, :,:,:]
# test_Masks= Masks_1[20000:21000, :,:,:]

test_Images.shape

In [ ]:
# Out of 34000 images , select 20,000 images for training
Images =Images[0:15000, :,:,:]
Masks=Masks[0:15000, :,:,:]

In [ ]:
Masks.shape

### Cross Validation with 5 k-Folds

In [ ]:
# Cross Validation: dividing data into 5 folds.
from sklearn.model_selection import KFold, StratifiedKFold
kf = KFold(n_splits = 2)

In [ ]:
# Generating 5 models base on cross validation sets. Save models in folder saved_models
VALIDATION_ACCURACY = []
VALIDAITON_LOSS = []

save_dir = 'saved_models_28/'
fold_var = 1
count=0
for train_index, val_index in kf.split(Images,Masks):
    training_data = Images[train_index]
    validation_data = Images[val_index]

    training_mask = Masks[train_index]
    validation_mask = Masks [val_index]
    
    model = sm.Unet(BACKBONE, encoder_weights='imagenet')
    model.compile(optimizer='adam',loss ='binary_crossentropy', metrics=['accuracy'])
    model_name= 'model_'+str(fold_var)+'.h5'
    checkpoint = tf.keras.callbacks.ModelCheckpoint(save_dir+model_name, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
    callbacks = [
             tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2),
              tf.keras.callbacks.TensorBoard(log_dir="logs"),checkpoint
        ]

    history = model.fit(x=training_data, y = training_mask, batch_size=32, epochs=3, verbose = 1, validation_data=(validation_data, validation_mask),callbacks=callbacks)	
    model.load_weights("saved_models_28/model_"+str(fold_var)+".h5")
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(1, len(loss) + 1)
    plt.plot(epochs, loss, 'y', label='Training loss')
    plt.plot(epochs, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()
    
    results = model.evaluate(validation_data,validation_mask)
    results = dict(zip(model.metrics_names,results))
    VALIDATION_ACCURACY.append(results['accuracy'])
    VALIDAITON_LOSS.append(results['loss'])
    tf.keras.backend.clear_session()
    fold_var += 1
    

##### Print the accuracy and loss of the cross validation.

In [ ]:
# Validation Accuracy for 5 models
VALIDATION_ACCURACY

In [ ]:
# Validation Loss for 5 models
VALIDAITON_LOSS

### Calculate IOU

In [ ]:
#Run this cell only if test size is small. otherwise it will give error. This cell works fine for test images equal to 1000
model = tf.keras.models.load_model('saved_models_28/model_1.h5')

test_pred = model.predict(test_Images, verbose=1)
test_pred_threshold = test_pred > 0.5

IOU_keras=tf.keras.metrics.MeanIoU(num_classes= 2)
IOU_keras.update_state(test_Masks, test_pred_threshold)
print('IOU of the base model is ',(IOU_keras.result().numpy()*100))

### Calculate Precision And Recall and Display output prediction

In [ ]:
model = tf.keras.models.load_model('saved_models_28/model_1.h5')
test_pred = model.predict(test_Images, verbose=1)
test_pred_threshold = test_pred > 0.5

Y_pred= test_pred_threshold.astype(np.uint16)
Y_val = test_Masks
Y_pred1= Y_pred.reshape(-1,1)
Y_val1= Y_val.reshape(-1,1)
np.unique(Y_val)
from sklearn.metrics import confusion_matrix
cf_matrix = confusion_matrix(Y_val1, Y_pred1)
print(cf_matrix)
sns.heatmap(cf_matrix/np.sum(cf_matrix), cmap="Reds", annot=True, fmt = '.2%', square=1,   linewidth=2.)
TP=cf_matrix[1][1]
FP=cf_matrix[0][1]
FN=cf_matrix[1][0]
TN=cf_matrix[0][0]

print("TN: ",TN," FP: ",FP," TP: ",TP," FN: ",FN)
precision = (TP/(TP+FP))*100
recall = (TP/(TP+FN))*100



print ("precision : ", precision , 'recall: ', recall)



In [ ]:
## Display the first 10 images
count = 0
for i in range(len(test_Images)):
  fig = plt.figure(figsize=(10, 7))
  rows,columns=1,3
  fig.add_subplot(rows, columns, 1)
  toShow=(1)*4  
  # showing image
  plt.imshow(test_Images[i])
  plt.axis('off')
  plt.title("Image")
  
    
  # Adds a subplot at the 2nd position
  fig.add_subplot(rows, columns, 2)
  plt.imshow(test_Masks[i][:,:,0])
  plt.axis('off')
  plt.title("Actual Mask")

  fig.add_subplot(rows, columns, 3)
  plt.imshow(test_pred_threshold[i][:,:,0])
  plt.axis('off')
  plt.title("Predicted Mask")
  count = count+1
  if count ==10:
     break
        
## ajouter une validation colorimétrique?

For example, we obtained: 

precision :  80.60307147076196 recall:  74.87873311774777

## Optional Morphology

In [ ]:
## Work in progress

from scipy import ndimage
from skimage import measure, color, io
import pandas as pd
from skimage.measure import label, regionprops, regionprops_table
from tensorflow.keras.preprocessing import image
pd.options.display.max_columns = None
counter=0

for i in test_pred_threshold:
    i=i.astype('uint8')
    uniqueValues,count = np.unique(i,return_counts=True)
    
    kernel = np.ones((1,20), np.uint8)  # note this is a horizontal kernel
    d_im = cv2.dilate(i[:,:,0], kernel, iterations=1)
    e_im = cv2.erode(d_im, kernel, iterations=1) 
    label_mask,num_labels=ndimage.label(e_im)
    num_labels
    print(num_labels)
    plot_comparison(i, e_im, 'e_im')
    clusters = measure.regionprops(label_mask, i)
    propList = ['major_axis_length',
            'minor_axis_length',
            'area',
            'equivalent_diameter', #Added... verify if it works
            'orientation', #Added, verify if it works. Angle btwn x-axis and major axis.
            'perimeter'
            ] 
    print(pd.DataFrame(regionprops_table(label_mask, i, properties=propList)))
    for j in range(num_labels):
        if (clusters[j].major_axis_length > (clusters[j].minor_axis_length)*9):
            label_mask[tuple(clusters[j].coords.T)]=0
        else:
            label_mask[tuple(clusters[j].coords.T)]=1
    
    label_mask = image.img_to_array(label_mask, dtype='uint8')
    label_mask[label_mask==1]= 255
    i[i==1]= 255
    uniqueValues,count = np.unique(label_mask,return_counts=True)
    uniqueValues,count = np.unique(i,return_counts=True)
    original = i.copy()
    final = cv2.bitwise_and(i,label_mask)
    updated[counter]=np.expand_dims(final, -1)
    counter = counter + 1
    
'''
# To Display updated patches after apply morphology
    fig = plt.figure(figsize=(10, 7))
    rows,columns=1,2
    fig.add_subplot(rows, columns, 1)
    toShow=(1)*4  
    # showing image
    plt.imshow(original, cmap='gray')
    plt.axis('off')
    plt.title("Original")

    # Adds a subplot at the 2nd position
    fig.add_subplot(rows, columns, 2)
    plt.imshow(i,cmap='gray')
    plt.axis('off')
    plt.title("Final Image")
    
    counter = counter+1
    if counter == 50:
        break
'''

## Generating data for next phase

By using the previous trained model

In [ ]:
# Predict test Images and generate data for next training phase. Save new data into folder New_Training_folder
model = tf.keras.models.load_model('saved_models/model_1.h5')
test_pred = model.predict(test_Images, verbose=1)
test_pred_threshold = test_pred > 0.6

# # Saving Files

Imagebase = 'New_Training_Data/Maps'
os.listdir(Imagebase )
Maskbase = 'New_Training_Data/Masks'
len(os.listdir(Imagebase ))

count = 0
for i in range(len(test_Images)):
    fig = plt.figure(figsize=(10, 7))
    rows,columns=1,3
    fig.add_subplot(rows, columns, 1)
    toShow=(1)*4  
    
    # showing image
    name= 'Image_'+str(i)+'.jpg'
    im_path = os.path.join(Imagebase, name)
    plt.imshow(test_Images[i])
    plt.axis('off')
    plt.title(name)

    
    cv2.imwrite (im_path,cv2.cvtColor(test_Images[i], cv2.COLOR_RGB2BGR))
    
      
    # Adds a subplot at the 2nd position
    fig.add_subplot(rows, columns, 2)
    plt.imshow(test_Masks[i][:,:,0])
    plt.axis('off')
    plt.title("Actual Mask")

    fig.add_subplot(rows, columns, 3)
    plt.imshow(test_pred_threshold[i][:,:,0])
    plt.axis('off')
    name= 'Mask_'+str(i)+'.jpg'
    plt.title(name)
  
    im_path = os.path.join(Maskbase, name)
    print(im_path)
    mask_to_save = test_pred_threshold[i][:,:,0].astype('float32')
    mask_to_save[mask_to_save>0]=255
    cv2.imwrite (im_path,mask_to_save)
    count = count+5


## Training Phase # 02

### Reading Custom Training Data

In [ ]:
count = 0
train_ids_paths=[]
train_masks_path=[]
base = 'New_Training_Data'


total_maps= os.listdir(base)
print(total_maps)
#print(os.listdir(base))
with tf.device('/gpu:0'):
    t = sorted(glob.glob(os.path.join(base, total_maps[2],'*.jpg')))
    m = sorted(glob.glob(os.path.join(base, total_maps[1],'*.jpg')))
    train_ids_paths +=t
    train_masks_path += m
print(len(train_ids_paths), len(train_masks_path))


In [ ]:
to_delete = sorted(glob.glob(os.path.join(base, total_maps[0],'*.jpg')))
len(to_delete)

In [ ]:
from tqdm import tqdm
Masks=[]
Images=[]  
count = 0 
with tf.device('/gpu:0'):     
  for img_path,mask_path in tqdm(zip(train_ids_paths, train_masks_path),total=len(train_ids_paths),position=0, leave=False):
    img = Image.open(img_path)
    Images.append(np.array(img))
    mask= cv2.imread(mask_path,2)
    mask= np.array(mask)
    mask = np.expand_dims(mask,-1)
    mask[mask<40]=0
    mask[mask>40]=1
    Masks.append(mask)
    count = count +1
print(len(Images), len(Masks))

In [ ]:
# transform the list into a Numpy array
Images=np.array(Images)
Masks=np.array(Masks)

In [ ]:
from sklearn.utils import shuffle
Images, Masks=shuffle(Images, Masks, random_state=42)

##### Display the first 5 images

In [ ]:
count = 0
for i in range(len(Images)):
  if count < 5:
      fig = plt.figure(figsize=(10, 7))
      rows,columns=1,2
      fig.add_subplot(rows, columns, 1)
      toShow=(1)*4  
      # showing image
      plt.imshow(Images[i])
      plt.axis('off')
      plt.title("Image")

      # Adds a subplot at the 2nd position
      fig.add_subplot(rows, columns, 2)
      plt.imshow(Masks[i][:,:,0], cmap='gray')
      plt.axis('off')
      plt.title("Mask")
  count +=1

print (count)

##### Select some images and train

In [ ]:
Images_1 = Images.copy()
Masks_1 = Masks.copy()

In [ ]:
# test_Images=Images_1[13000:14001, :,:,:]
# test_Masks= Masks_1[13000:14001, :,:,:]

test_Images=Images_1[9000:10000, :,:,:]
test_Masks= Masks_1[9000:10000, :,:,:]

test_Images.shape

In [ ]:
Images =Images[0:800, :,:,:]
Masks=Masks[0:800, :,:,:]

In [ ]:
from sklearn.model_selection import KFold, StratifiedKFold

kf = KFold(n_splits = 2)

In [ ]:
VALIDATION_ACCURACY = []
VALIDAITON_LOSS = []

save_dir = 'custom_saved_models/'
fold_var = 1
count=0
for train_index, val_index in kf.split(Images,Masks):
    training_data = Images[train_index]
    validation_data = Images[val_index]

    training_mask = Masks[train_index]
    validation_mask = Masks [val_index]
    
    model = sm.Unet(BACKBONE, encoder_weights='imagenet')
    model.compile(optimizer='adam',loss ='binary_crossentropy', metrics=['accuracy'])
    model_name= 'model_'+str(fold_var)+'.h5'
    checkpoint = tf.keras.callbacks.ModelCheckpoint(save_dir+model_name, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
    callbacks = [
             tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2),
              tf.keras.callbacks.TensorBoard(log_dir="logs"),checkpoint
        ]

    history = model.fit(x=training_data, y = training_mask, batch_size=32, epochs=3, verbose = 1, validation_data=(validation_data, validation_mask),callbacks=callbacks)	
    model.load_weights("custom_saved_models/model_"+str(fold_var)+".h5")
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(1, len(loss) + 1)
    plt.plot(epochs, loss, 'y', label='Training loss')
    plt.plot(epochs, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()
    
    results = model.evaluate(validation_data,validation_mask)
    results = dict(zip(model.metrics_names,results))
    VALIDATION_ACCURACY.append(results['accuracy'])
    VALIDAITON_LOSS.append(results['loss'])
    tf.keras.backend.clear_session()
    fold_var += 1

Print loss and accuracy in the notebook.

In [ ]:
VALIDATION_ACCURACY

In [ ]:
VALIDAITON_LOSS

Training Phase 2 Completed and stored in Custom_saved_model folder

## Comparing Results of Custom Trained Model vs Base Trained Model

In [ ]:
# Loading base model
model = tf.keras.models.load_model('saved_models_28/model_1.h5')

# Loading custom model
model_custom = tf.keras.models.load_model('custom_saved_models/model_1.h5')


#Prediction Using Base Model
test_pred = model.predict(test_Images, verbose=1)
test_pred_threshold = test_pred > 0.5

#Prediction Using Custom Model
test_pred_custom = model_custom.predict(test_Images, verbose=1)
test_pred_threshold_custom = test_pred_custom > 0.5

In [ ]:
#Run only if test data is equal or less than 1000 images, otherwise GPU error will come
IOU_keras=tf.keras.metrics.MeanIoU(num_classes= 2)

#IOU using Base Model
IOU_keras.update_state(test_Masks, test_pred_threshold)
print('IOU using BAse Model is ',(IOU_keras.result().numpy()*100))

#IOU using Custom Model
IOU_keras.update_state(test_Masks, test_pred_threshold_custom)
print('IOU using Custome Model is ',(IOU_keras.result().numpy()*100))



### Precision and Recall using Base Model

In [ ]:
# Precision Recall using Base Model
Y_pred= test_pred_threshold.astype(np.uint16)
Y_val = test_Masks
Y_pred1= Y_pred.reshape(-1,1)
Y_val1= Y_val.reshape(-1,1)
np.unique(Y_val)
from sklearn.metrics import confusion_matrix
cf_matrix = confusion_matrix(Y_val1, Y_pred1)
print(cf_matrix)
sns.heatmap(cf_matrix/np.sum(cf_matrix), cmap="Reds", annot=True, fmt = '.2%', square=1,   linewidth=2.)
TP=cf_matrix[1][1]
FP=cf_matrix[0][1]
FN=cf_matrix[1][0]
TN=cf_matrix[0][0]


print("*"*100)
print("Base Model: Results and Statistics")
print("*"*100)

print("TN: ",TN," FP: ",FP," TP: ",TP," FN: ",FN)
precision = (TP/(TP+FP))*100
recall = (TP/(TP+FN))*100
print ("precision : ", precision , 'recall: ', recall)


In [ ]:
## Display the first 10 images
count = 0
for i in range(len(test_Images)):
  fig = plt.figure(figsize=(10, 7))
  rows,columns=1,3
  fig.add_subplot(rows, columns, 1)
  toShow=(1)*4  
  # showing image
  plt.imshow(test_Images[i])
  plt.axis('off')
  plt.title("Image")
  
    
  # Adds a subplot at the 2nd position
  fig.add_subplot(rows, columns, 2)
  plt.imshow(test_Masks[i][:,:,0])
  plt.axis('off')
  plt.title("Actual Mask")

  fig.add_subplot(rows, columns, 3)
  plt.imshow(test_pred_threshold[i][:,:,0])
  plt.axis('off')
  plt.title("Predicted Mask")
  count = count+1
  if count ==10:
     break

### Precision and Recall using Custom Model

In [ ]:
# Precision Recall using Custom Model
# Precision Recall using Base Model

model = tf.keras.models.load_model('custom_saved_models/model_1.h5')
test_pred = model.predict(test_Images, verbose=1)
test_pred_threshold = test_pred > 0.5

Y_pred= test_pred_threshold.astype(np.uint16)
Y_val = test_Masks
Y_pred1= Y_pred.reshape(-1,1)
Y_val1= Y_val.reshape(-1,1)
np.unique(Y_val)
from sklearn.metrics import confusion_matrix
cf_matrix = confusion_matrix(Y_val1, Y_pred1)
print(cf_matrix)
sns.heatmap(cf_matrix/np.sum(cf_matrix), cmap="Reds", annot=True, fmt = '.2%', square=1,   linewidth=2.)
TP=cf_matrix[1][1]
FP=cf_matrix[0][1]
FN=cf_matrix[1][0]
TN=cf_matrix[0][0]

print("*"*100)
print("Custom Model: Results and Statistics")
print("*"*100)

print("TN: ",TN," FP: ",FP," TP: ",TP," FN: ",FN)
precision = (TP/(TP+FP))*100
recall = (TP/(TP+FN))*100


print ("precision : ", precision , 'recall: ', recall)


In [ ]:
## Display the first 10 images
count = 0
for i in range(len(test_Images)):
  fig = plt.figure(figsize=(10, 7))
  rows,columns=1,3
  fig.add_subplot(rows, columns, 1)
  toShow=(1)*4  
  # showing image
  plt.imshow(test_Images[i])
  plt.axis('off')
  plt.title("Image")
  
    
  # Adds a subplot at the 2nd position
  fig.add_subplot(rows, columns, 2)
  plt.imshow(test_Masks[i][:,:,0])
  plt.axis('off')
  plt.title("Actual Mask")

  fig.add_subplot(rows, columns, 3)
  plt.imshow(test_pred_threshold[i][:,:,0])
  plt.axis('off')
  plt.title("Predicted Mask")
  count = count+1
  if count ==10:
     break

## PATCHIFY CODE START HERE
Test on one single image using the custom trained model.

### Read Image File

In [ ]:
model = tf.keras.models.load_model('custom_saved_models/model_1.h5')
#model = tf.keras.models.load_model('saved_models_28/model_1.h5')

In [ ]:
large_image = cv2.imread('OSD64/MAP_osd_64_berwickstjohn_1808.tif', 1)
print(large_image.shape)
large_image = cv2.cvtColor(large_image, cv2.COLOR_RGB2BGR) 
plt.imshow(large_image.astype('uint8'))
#plt.imshow(np.real(large_image))
Large_Image=np.array(large_image)


In [ ]:
large_mask = cv2.imread('OSD64/MASK_osd_64_berwickstjohn_1808.TIF', 0)
print(large_mask.shape)
# large_mask = cv2.cvtColor(large_image, cv2.COLOR_RGB2BGR) 
# plt.imshow(large_image.astype('uint8'))
plt.imshow(np.real(large_mask),'gray')
Large_mask=np.array(large_mask)
Large_mask=np.expand_dims(Large_mask, axis=-1)
print(Large_mask.shape)

### Create Container to hold Image for proper step size

In [ ]:
print('original imag shape',Large_Image.shape)
remainderW =  (Large_Image.shape[0] - 256) % 256
remainderH =  (Large_Image.shape[1] - 256) % 256

if remainderW != 0:
    width= Large_Image.shape[0] -remainderW +256
else:
    width = Large_Image.shape[0]

if remainderH != 0:
    height= Large_Image.shape[1] -remainderH +256
else:
    height = Large_Image.shape[1]
    
print(width , height )

container = np.zeros((width, height,3), dtype=int)
container.shape
container[0:Large_Image.shape[0], 0:Large_Image.shape[1],:] = Large_Image[0:Large_Image.shape[0],0:Large_Image.shape[1],:]
plt.imshow(container)
print('New image shape', container.shape)

### Create Patches

In [ ]:
patches= patchify(container, (256 ,256,3),step=256)
patches.shape
patches1=patches.reshape(patches.shape[0]*patches.shape[1]*patches.shape[2], 256,256, 3)
patches1.shape

### Perform Prediction on Patches


In [ ]:
test_pred = model.predict(patches1, verbose=1)
test_pred_threshold_03 = test_pred > 0.3
test_pred_threshold_04 = test_pred > 0.4
test_pred_threshold_05 = test_pred > 0.5
test_pred_threshold_06 = test_pred > 0.6
test_pred_threshold_07 = test_pred > 0.7
test_pred_threshold_08 = test_pred > 0.8
test_pred_threshold_09 = test_pred > 0.9


### Final Changing - Consensus 

In [ ]:
final_pred= np.zeros((1,55050240))
pred3 = test_pred_threshold_03.reshape(1,-1)
pred4 = test_pred_threshold_04.reshape(1,-1)
pred5 = test_pred_threshold_05.reshape(1,-1)
pred6 = test_pred_threshold_06.reshape(1,-1)
pred7 = test_pred_threshold_07.reshape(1,-1)
pred8 = test_pred_threshold_08.reshape(1,-1)
pred9 = test_pred_threshold_09.reshape(1,-1)
flag_on=0
flag_off=0

for i in range(len(pred3[0])):
    flag_on=0
    flag_off=0
    for j in range(3,10):
        test_pred=locals()['pred'+str(j)]
        if (test_pred[0][i] == True):
            flag_on+=1
        else:
            flag_off+=1
    if (flag_on>flag_off+2):
        # cela a été décidé un peu au "pif" ... peut-être cela peut être revu ou justifié.
        
        final_pred[0][i]=1
    else:
        final_pred[0][i]=0
final_pred=final_pred.reshape((840, 256, 256, 1))

### Merge Patches

In [ ]:
test_pred_threshold=final_pred
#print(test_pred_threshold.shape , patches1.shape)
merge_patches= patches1.reshape(patches.shape[0],patches.shape[1],patches.shape[2], 256,256, 3)
merge_mask = test_pred_threshold.reshape(patches.shape[0],patches.shape[1],patches.shape[2], 256,256, 1)
merge_patches=unpatchify(merge_patches,container.shape)
merge_masks=unpatchify(merge_mask,(container.shape[0],container.shape[1],1))
merge_masks = merge_masks.astype(int)
merge_masks[merge_masks==1]=255
np.unique(merge_masks)
#plt.imshow(merge_masks, 'gray')
print(merge_masks.shape)
merge_masks = merge_masks[0:Large_Image.shape[0], 0:Large_Image.shape[1],:]
print(merge_masks.shape, Large_Image.shape)
plt.imshow(merge_masks, 'gray')

In [ ]:
merge_masks[merge_masks>40]=255
merge_masks[merge_masks<40]=0
gt_path='OSD64'
img='MASK_osd_64_berwickstjohn_1808'
name ='Consensus_2 '+img+'_new_model_pred_'+str(i+1)+'.png'
im_path = os.path.join(gt_path, name)
print(im_path)
cv2.imwrite (im_path,merge_masks)
#test_pred = model.predict(patches1, verbose=1)

### Calculate Statistics

In [ ]:
#Calculate Stats

Y_pred= merge_masks
Y_val = Large_mask
Y_pred[Y_pred>0]=1
Y_val[Y_val>0]=1
Y_pred1= Y_pred.reshape(-1,1)
Y_val1= Y_val.reshape(-1,1)


from sklearn.metrics import confusion_matrix
cf_matrix = confusion_matrix(Y_val1, Y_pred1)
TP=cf_matrix[1][1]
FP=cf_matrix[0][1]
FN=cf_matrix[1][0]
TN=cf_matrix[0][0]

print("TN: ",TN," FP: ",FP," TP: ",TP," FN: ",FN)
precision = (TP/(TP+FP))*100
recall = (TP/(TP+FN))*100


print ("precision : ", precision , 'recall: ', recall)
    
   


### PATCHIFY CODE END HERE